# Sudoku Board coding

### Author: Umberto Michelucci (C) 2017 

This notebook contains code that can solve any Sudoku with a unique solution in Python. It is written in Python 3 and don't contain any special packages, so you should be able to run it with a pretty standard Python installation (Anaconda for example). It is loosely based on the first project in the Udacity AI Nanodegree. 

There is a nice discussion on the mathematics behind the Sudoku here: http://www.math.cornell.edu/~mec/Summer2009/Mahmood/Home.html

We first start by creating vectors and matrices that contains all the cell names. We will code the name giving to each row a letter and to each column a number.

In [21]:
rows = 'ABCDEFGHI'
cols = '123456789'

We need a helper function, `cross(a, b)`, which, given two strings — a and b — will return the list formed by all the possible concatenations of a letter s in string a with a letter t in string b. In this way we can get each box separately.

In [22]:
def cross(a, b):
      return [s+t for s in a for t in b]

Let's test it with our rows and cols. Each position in our Sudoku board is called a box.
Our cross() function will return all the boxes moving from the top to the right, giving the first row, and then passing to the subsequents rows. So it will start with A1 until A9, then go to the second row and give B1 until B9 and so on.

Check the image below to understand how each box is called.

![](sudoku_board_1.png)

In [23]:
boxes = cross(rows,cols)
boxes[0:10] # First 10 boxes

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1']

Now let's find the rows, cols and units. We will need to have dictionaries, beacuse we will use them extensively to find all the peers of a specific unit.

In [24]:
row_units = [cross(r, cols) for r in rows]
# For example:
# row_units[0] = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']

column_units = [cross(rows, c) for c in cols]
# For example:
# column_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']

square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
# For example:
# square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

unitlist = row_units + column_units + square_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

Note that with peers of a specific box, we intend all the units that are in a unit (the 3x3 box that contains the box), in the column of the box and in the row of the box.

In the image all the peers of box B5 are colored in green.

![](sudoku_peers.png)

Let's check the peers of box "B5". Remember `peers` is a dictionary.

In [58]:
peers["B5"]

{'A4',
 'A5',
 'A6',
 'B1',
 'B2',
 'B3',
 'B4',
 'B6',
 'B7',
 'B8',
 'B9',
 'C4',
 'C5',
 'C6',
 'D5',
 'E5',
 'F5',
 'G5',
 'H5',
 'I5'}

# Representation of the puzzle

How we can represent a puzzle? The easiest way of doing that is to prepare a string, 81 characters long, with each character representing a box and the digit value the digit appearing in the box itself. We will go from top left to the right and then down. So we will start from "A1" and go until "A9", then pass to "B1" and continues to "B9" and so on. To make an example the following puzzle

![](sudoku_puzzle1.png)

Will have the following representation

In [60]:
puzzle = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

Now we will need to helper functions. One that will display the puzzle (for simplicity) and one to transform this String representation into a dictionary. This will make things later on a lot easier.

## Helper Functions

The next function (`display()`) will display the puzzle in ASCII, making it easy to display results and temporary steps. The function has been taken from code that is coming from Udacity.

In [26]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)


The next function we need is a function that converts the string representation into a dictionary. The first function (`grid_values()`) will replace "." with "123456789", with the idea that where there is an empty cell any digit could appear. This make visualisation good to test partial solutions, but not that good to visualize the puzzle itself. So I have a slightly modified version of the function that will keep the "." and not replace it with "123456789" (`grid_values_empty()`).

In [61]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = list(grid)
    values = [x if x != '.' else '123456789' for x in values]
    return (dict(zip(boxes, values)))

def grid_values_empty(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    values = list(grid)
    #values = [x if x != '.' else '.' for x in values]
    return (dict(zip(boxes, values)))

Now let's try our helper functions

In [62]:
sudoku1 = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
display(grid_values(sudoku1))

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

And we can display it just with `.` where there is not a digit. Is not useful to our algorithm, but it is useful to humans to visualize it.

In [50]:
display(grid_values_empty(sudoku1))

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


This is a quite easy Sudoku. We will try harder ones at the end when we have our algorithm coded.

# Function to eliminate digits from peers of each box with a single value

If a box has a single value in it, we can eliminate that digits to all the peers of that box. Simplifying our puzzle. The function `eliminate()` will do exactly this. This is an example of **constraint propagration**.

In [40]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    for box,value in values.items():
        if len(value) == 1:
            # Remove now the value from all the peers. 
            # So the first thing to do is to loop over all the peers,
            # being rows, cols or units
            for peer in peers[box]:
                values[peer] = values[peer].replace(value, "")
    return values

Remember that Python works, unless you specify otherwise, with references. So the moment you call the function `eliminate()` you will modify your initial dictionary. So keep that in mind. This is also the reason why you will find later on some deepcopy commands in the code.

In [76]:
my_initial_sudoku = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')

In [77]:
from copy import deepcopy

In [78]:
my_initial_sudoku
my_second_sudoku = deepcopy(my_initial_sudoku)
my_second_sudoku = eliminate(my_second_sudoku)
display(my_second_sudoku)

   45    4578    3   |   9      2      14  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |   59   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6      8      9   |   5     1478    47  
   8     1467    47  |   2      5      3   |   17    1467    9   
   6      69     5   |   4      1      7   |   3     268     26  


And we see how our function has already simplified our puzzle enormously! 

One call to the `eliminate()` function was not enough to solve the puzzle. We will need to work on it a bit more. But we were already able to reduce quite a lot the number of digits we have in the puzzle.

This is a simple puzzle, and it can be solved just by applying `eliminate()` several times. You can try and you will see how you get to a solution. Not all sudokus will be so easy. 

# Function to finalise all values that are the only choice for a box

The function we need now, is one that goes through all units, and if in one there a value that fits only in one box, assign the value to this box. This is another example of **constraint propagration**.

In [73]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    # TODO: Implement only choice strategy here
    for box,value in values.items():
        
        # Now let's iterate over all digits 
        # and see if it appears only once
        if len(value) > 1: # We only check when the number of digits is > 1
            for c in value:
                for peer_list in units[box]:
                    sm = 0
                    for bx in peer_list:
                        if c in values[bx]:
                            sm += 1
                    if (sm == 1):
                        values[box] = values[box].replace(value, c)
                
    return values


In [79]:
display(my_initial_sudoku)

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

In [80]:
tmp = deepcopy(my_second_sudoku)
display(my_second_sudoku)


   45    4578    3   |   9      2      14  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |   59   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6      8      9   |   5     1478    47  
   8     1467    47  |   2      5      3   |   17    1467    9   
   6      69     5   |   4      1      7   |   3     268     26  


In [81]:
tmp2 = deepcopy(tmp)
display(only_choice(tmp2))

  45   4578   3   |  9     2     1   |  6    5789   57  
  9     6     47  |  3     4     5   |  8     2     1   
  2    257    1   |  8     7     6   |  4     9     3   
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9   34567 34567 
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5    1478   47  
  8    1467   47  |  2     5     3   |  7    1467   9   
  6     9     5   |  4     1     7   |  3     8     2   


And as you can check we have simplifed further our puzzle, but we are not yet at the solution. 

# Iteration of the two previous steps

Now what we can do is we can iterate our previous steps several times to try to get to a solution. We will need to put some check to know if we are at a solution or we stalled. In both case we need to stop. The function `reduce_puzzle()` will do exactly this. This is a **constraint propagation** example. We do it several times since after each iteration our puzzle changes and we can simplify it a bit more.

In [85]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        eliminate(values)
        only_choice(values)
        
        # Are we finished yet?
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        # Note: this will happen when using the search method. While searching
        # for a solution, in a case where there are no solutions, what will happen
        # is that the eliminate() function will remove all the digits from a box, telling
        # us that there is no solution. This will be useful when writing the recursive function
        # search()
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values


Let's try now to use our function and see if we get to a solution. Let's consider the following puzzle

In [82]:
tmp = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
display(grid_values_empty('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


In [86]:
display(reduce_puzzle(tmp2))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


In this case we get a nice solution immediately, without the need of implementing more sophisticated algorithm (we will need for more difficult puzzles, that is what we will do in the next sections).

# Most difficult Sudoku - Can only constraint propagation solve it?

Now we can get what is considered the most difficult Sudoku (http://www.conceptispuzzles.com/de/index.aspx?uri=info/article/424) and try to apply our **constraing propagration** and see if we can solve it.

In [90]:
sud = grid_values('8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..')
display(sud)

    8     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789     3     |    6     123456789 123456789 |123456789 123456789 123456789 
123456789     7     123456789 |123456789     9     123456789 |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     5     123456789 |123456789 123456789     7     |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789     4         5     |    7     123456789 123456789 
123456789 123456789 123456789 |    1     123456789 123456789 |123456789     3     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     1     |123456789 123456789 123456789 |123456789     6         8     
123456789 123456789     8     |    5     123456789 123456789 |123456789     1     123456789 
123456789     9     123456789 |123456789 123456789 123456789 |    4   

We can display it with "." where there are no digits.

In [91]:
display(grid_values_empty('8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..'))

8 . . |. . . |. . . 
. . 3 |6 . . |. . . 
. 7 . |. 9 . |2 . . 
------+------+------
. 5 . |. . 7 |. . . 
. . . |. 4 5 |7 . . 
. . . |1 . . |. 3 . 
------+------+------
. . 1 |. . . |. 6 8 
. . 8 |5 . . |. 1 . 
. 9 . |. . . |4 . . 


In [89]:
display(reduce_puzzle(sud))

   8      1246   24569  |  2347   12357    1234  | 13569    4579  1345679 
 12459    124      3    |   6     12578    1248  |  1589   45789   14579  
  1456     7      456   |  348      9      1348  |   2      458    13456  
------------------------+------------------------+------------------------
 123469    5      2469  |  2389    2368     7    |  1689    2489   12469  
 12369   12368    269   |  2389     4       5    |   7      289     1269  
 24679    2468   24679  |   1      268     2689  |  5689     3     24569  
------------------------+------------------------+------------------------
 23457    234      1    | 23479    237     2349  |  359      6       8    
 23467    2346     8    |   5      2367   23469  |   39      1      2379  
 23567     9      2567  |  2378   123678  12368  |   4      257     2357  


As you can see, we don't get a solution. In many boxes there are several values that are possible. So we need to implement a more sophisticated algorithm (in a particular a search algorithm). 

# A search algorithm

The idea to solve the puzzle is to choose a box with more than one digit in it. Choose one of the digit and assume that in that box there is only _that_ digit and try to solve the puzzle. If that does not work try with the other digits.

For example for the puzzle given above, you could choose box "H7" where only the digits 39 are available. So one could assume that in "H7" there is a 3 and try to solve the puzzle with our **constraint propagration** functions. We need to write a recursive function, since what will happen is that all the possibilities are structured as a tree, and we need to go over all the branches to find a solution. 

The next function is doing exactly that.

In [94]:
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [95]:
display(search(sud))

8 1 2 |7 5 3 |6 4 9 
9 4 3 |6 8 2 |1 7 5 
6 7 5 |4 9 1 |2 8 3 
------+------+------
1 5 4 |2 3 7 |8 9 6 
3 6 9 |8 4 5 |7 2 1 
2 8 7 |1 6 9 |5 3 4 
------+------+------
5 2 1 |9 7 4 |3 6 8 
4 3 8 |5 2 6 |9 1 7 
7 9 6 |3 1 8 |4 5 2 


As you can see this method will solve the puzzle and can solve all puzzles that have a unique solution. And by the way in "H7" there is a 9 not a 3 ;-)

As a side note for a Sudoku to have a unique solution it must have 17 hints (check this link from MIT https://www.technologyreview.com/s/426554/mathematicians-solve-minimum-sudoku-problem/). With less you will get multiple solutions.

# A Sudoku with more than one solution

There is a nice example of a Sudoku with more than one solution (check http://sandwalk.blogspot.ch/2007/06/i-knew-it-there-can-be-more-than-one.html). Let's see what our method find...

In [92]:
sud_multiple = "9.6.7.4.3...4..2...7..23.1.5.....1...4.2.8.6...3.....5.3.7...5...7..5...4.5.1.7.8"
display(grid_values_empty(sud_multiple))

9 . 6 |. 7 . |4 . 3 
. . . |4 . . |2 . . 
. 7 . |. 2 3 |. 1 . 
------+------+------
5 . . |. . . |1 . . 
. 4 . |2 . 8 |. 6 . 
. . 3 |. . . |. . 5 
------+------+------
. 3 . |7 . . |. 5 . 
. . 7 |. . 5 |. . . 
4 . 5 |. 1 . |7 . 8 


In [97]:
display(search(grid_values(sud_multiple)))

9 2 6 |5 7 1 |4 8 3 
3 5 1 |4 8 6 |2 7 9 
8 7 4 |9 2 3 |5 1 6 
------+------+------
5 8 2 |3 6 7 |1 9 4 
1 4 9 |2 5 8 |3 6 7 
7 6 3 |1 4 9 |8 2 5 
------+------+------
2 3 8 |7 9 4 |6 5 1 
6 1 7 |8 3 5 |9 4 2 
4 9 5 |6 1 2 |7 3 8 


We find a solution of course, but there is another solution (due to symmetry). Check the link to see the other one. the trick is that our method goes through all the branches and then stops as soon as it finds one. So the method will not notice that the method has two solutions. To see all possible solutions one should memorize the solution and continue to search until all possibilities are exhausted.